In [ ]:
import os
import pandas as pd
import seaborn as sns

In [ ]:
data_location = '..\data\endsong_'

# read first file to get df format
index = 0
df = pd.read_json(f'{data_location}{index}.json')
index += 1

# append the rest of the files
while os.path.exists(f'{data_location}{index}.json'):
    df = pd.concat([df, pd.read_json(f'{data_location}{index}.json')])
    index += 1

remove tracks not listened completely

In [ ]:
df = df[(df.reason_end == 'trackdone') & (df.ms_played > 0)]

parse and offset datetime

In [ ]:
df['ts'] = pd.to_datetime(df.ts, utc=True)
df.ts = df.ts.dt.tz_convert('America/Argentina/Cordoba')

## categorical

In [ ]:
# categorical labels
df['weekday'] = df.ts.map(lambda x: x.weekday())
df['time'] = df.ts.map(lambda x: f'{x.hour:02}:{(x.minute // 30) * 30:02}') # groups of 30 minutes

### time x weekday

In [ ]:
grouping = ['time', 'weekday']
grouping_label = ' '.join(grouping)

grouped = df
grouped[grouping_label] = grouped.apply(lambda x: ' '.join([str(x[grouping[0]]), str(x[grouping[1]])]), axis=1)

group and count

In [ ]:
frequencies = grouped[[grouping_label, 'ts']].groupby(grouping_label).count()

frequencies[grouping[0]] = frequencies.index.map(lambda x: x.split(' ')[0])
frequencies[grouping[1]] = frequencies.index.map(lambda x: ' '.join(x.split(' ')[1:]))

In [ ]:
pivot = pd.pivot_table(frequencies, values='ts', index=[grouping[1]], columns=[grouping[0]], fill_value=0)
pivot

In [ ]:
sns.set(rc={'figure.figsize':(30,20)})
sns.heatmap(pivot)